# **AlexNet with CIFAR-10**

In [ ]:
"""
1. 必要なコアライブラリのインポート

torch: PyTorchのコアライブラリ。
torchvision: 画像処理用のデータセットやモデルを提供。
torch.nn: ニューラルネットワーク構築用のモジュール。
torch.optim: 最適化アルゴリズムを提供。
"""
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim


"""
2. CIFAR-10データセットの読み込み

CIFAR-10: 10クラスのカラー画像データセット（32x32ピクセル）。

画像に前処理を行って汎化性能を向上させる（テスト画像はオリジナルのまま）。
tf.RandomHorizontalFlip(p=0.5)
50%の確率で画像を水平反転。
tf.RandomRotation(degrees=15)
画像を-15度から+15度の範囲でランダムに回転。
tf.RandomAffine(degrees=0, translate=(0.1, 0.1))
平行移動を加える。translate=(0.1, 0.1)は画像を幅と高さの±10%以内でランダムに平行移動。
tf.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
画像のピクセル値を正規化（値を-1～1の範囲にスケーリング）。
transform=tf.ToTensor(): 画像データをTensor形式に変換。

download=True: データセットがローカルにない場合に自動的にダウンロード。

torchvision.transforms: データ前処理（変換）のユーティリティ。
"""
import torchvision.transforms as tf

train_transform = tf.Compose([
    tf.RandomHorizontalFlip(p=0.5),
    tf.RandomRotation(degrees=15),
    tf.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    tf.ToTensor(),
    tf.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transform = tf.Compose([
    tf.ToTensor(),
    tf.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = torchvision.datasets.CIFAR10(
    root='./data/',
    train=True,
    transform=train_transform,
    download=True)

test_dataset = torchvision.datasets.CIFAR10(
    root='./data/',
    train=False,
    transform=test_transform,
    download=True)


"""
3. データの大きさを確認

データセットのサイズ（画像の数）と、1つのサンプル画像の形状（[チャンネル, 高さ, 幅]）を表示。
"""
print ('train_data = ', len(train_dataset))
print ('test_data = ', len(test_dataset))
image, label = train_dataset[0]
print (image.size())


"""
4. DataLoaderの設定

DataLoader: データセットをミニバッチに分けて効率的にロード。
batch_size: ミニバッチのサイズ。
shuffle=True: トレーニングデータをランダムにシャッフル。
num_workers: データローディングに使用するスレッド数。
"""
train_loader = torch.utils.data.DataLoader(
      dataset=train_dataset,
      batch_size=64,
      shuffle=True,
      num_workers=2)

test_loader = torch.utils.data.DataLoader(
      dataset=test_dataset,
      batch_size=64,
      shuffle=False,
      num_workers=2)


"""
5. AlexNetモデルの定義

特徴抽出部 (self.features):

畳み込み層（Conv2d）とプーリング層（MaxPool2d）を組み合わせ、入力画像から特徴を抽出。
活性化関数としてReLUを使用。
畳み込みカーネルのサイズやパディングを変更し、AlexNetの簡略化バージョン。
分類部 (self.classifier):

全結合層（Linear）で抽出した特徴を10クラスに分類。
ドロップアウトで過学習を防ぐ。
forwardメソッド:

順伝播計算を定義。
特徴抽出後、4次元テンソルを平坦化して分類部に渡す。
"""
class AlexNet(nn.Module):

    def __init__(self, num_classes):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 4 * 4, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 4 * 4)
        x = self.classifier(x)
        return x


"""
6. デバイス設定とモデル構築

cuda: GPUが利用可能な場合、GPUで計算を実行。
to(device): モデルを指定デバイスに移動。
"""
num_classes = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'
neuralnet = AlexNet(num_classes).to(device)


"""
7. 損失関数と最適化手法

CrossEntropyLoss: クラス分類問題用の損失関数。
SGD: 確率的勾配降下法。学習率、モーメンタム、重み減衰（L2正則化）を設定。
"""
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(neuralnet.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)


"""
8. トレーニングと評価

trainモード:
モデルをトレーニングモードに設定し、順伝播→損失計算→逆伝播→パラメータ更新を実行。
ミニバッチごとに損失と正解率を計算。

valモード:
モデルを評価モードに設定（勾配計算をオフ）。
テストデータセットを使用して損失と正解率を評価。

平均値:
エポックごとの平均損失と平均正解率を計算して保存。
"""
num_epochs = 20
train_loss_list, train_acc_list, val_loss_list, val_acc_list = [], [], [], []

for epoch in range(num_epochs):
    train_loss, train_acc, val_loss, val_acc = 0, 0, 0, 0

    # training mode
    neuralnet.train()
    for i, (images, labels) in enumerate(train_loader):
      images, labels = images.to(device), labels.to(device)
      optimizer.zero_grad()
      outputs = neuralnet(images)
      loss = criterion(outputs, labels)
      train_loss += loss.item()
      train_acc += (outputs.max(1)[1] == labels).sum().item()
      loss.backward()
      optimizer.step()

    avg_train_loss = train_loss / len(train_loader.dataset)
    avg_train_acc = train_acc / len(train_loader.dataset)

    # val mode
    neuralnet.eval()
    with torch.no_grad():
      for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = neuralnet(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()
        val_acc += (outputs.max(1)[1] == labels).sum().item()
    avg_val_loss = val_loss / len(test_loader.dataset)
    avg_val_acc = val_acc / len(test_loader.dataset)

    print ('Epoch [{}/{}], Train_loss: {loss:.6f}, Validation_loss: {val_loss:.6f}'
                   .format(epoch+1, num_epochs, i+1, loss=avg_train_loss, val_loss=avg_val_loss))
    train_loss_list.append(avg_train_loss)
    train_acc_list.append(avg_train_acc)
    val_loss_list.append(avg_val_loss)
    val_acc_list.append(avg_val_acc)


"""
9. 学習過程の可視化

損失の可視化:
トレーニング損失と検証損失をエポックごとにプロット。

正解率の可視化:
トレーニング正解率と検証正解率をエポックごとにプロット。

matplotlib.pyplot: 可視化用ライブラリ。
"""
from matplotlib import pyplot as plt

plt.figure()
plt.plot(range(num_epochs), train_loss_list, color='red', linestyle='-', linewidth=2, label='train_loss')
plt.plot(range(num_epochs), val_loss_list, color='green', linestyle='--', linewidth=2, label='validation_loss')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Loss')
plt.grid()
plt.xticks(range(num_epochs))

plt.figure()
plt.plot(range(num_epochs), train_acc_list, color='red', linestyle='-', linewidth=2, label='train_accuracy')
plt.plot(range(num_epochs), val_acc_list, color='green', linestyle='--', linewidth=2, label='validation_accuracy')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('Accuracy')
plt.grid()
plt.xticks(range(num_epochs))

# **学習済みモデルによるテスト画像へのラベリング**

In [ ]:
"""
10. 学習済みモデルによるテスト画像へのラベリング

np.random.choiceでランダムに10枚選択

テストデータの中からランダムに10枚を選ぶ
torch.softmaxで確率計算

出力（ロジット）を確率に変換する
probabilities.argmax(dim=1)で推定ラベル取得

各画像について最も確率が高いクラスを推定ラベルとする
matplotlibで画像と結果をプロット

選ばれた画像を表示し、推定ラベル、正解ラベル、推定確率をプロット
データの正規化を元に戻す

画像を可視化するために(image * 0.5) + 0.5を行い、元のピクセル値（[0, 1]の範囲）に戻す

各ラベル番号は以下のクラスを意味する。
(0)airplanes, (1)cars, (2)birds, (3)cats, (4)deer, (5)dogs, (6)frogs, (7)horses, (8)ships, and (9)trucks

"""

import numpy as np

# ランダムに10枚の画像を選んで推定と表示を行う関数
def show_predictions(model, test_loader, num_samples=10):
    model.eval()  # モデルを評価モードに設定
    images, labels = next(iter(test_loader))  # テストデータから1バッチ分取得
    indices = np.random.choice(len(images), num_samples, replace=False)  # ランダムに画像を選択
    selected_images = images[indices]
    selected_labels = labels[indices]

    with torch.no_grad():  # 推論時は勾配計算を無効化
        selected_images = selected_images.to(device)
        outputs = model(selected_images)
        probabilities = torch.softmax(outputs, dim=1)  # 各クラスの確率
        predicted_classes = probabilities.argmax(dim=1)  # 推定ラベル

    # 画像と結果を表示
    plt.figure(figsize=(15, 5))
    for i in range(num_samples):
        plt.subplot(1, num_samples, i + 1)
        image = selected_images[i].cpu().numpy().transpose((1, 2, 0))  # 画像を[N, C, H, W]から[H, W, C]に変換
        image = (image * 0.5) + 0.5  # 正規化を元に戻す（[-1, 1] -> [0, 1]）
        plt.imshow(image)
        plt.title(f"Label: {selected_labels[i].item()}\nPred: {predicted_classes[i].item()}\nProb: {probabilities[i][predicted_classes[i]].item():.2f}")
        plt.axis('off')
    plt.show()

# テスト画像で推定結果を表示
show_predictions(neuralnet, test_loader, num_samples=10)

# **分類に貢献した特徴領域の可視化（Grad-CAM）**

In [ ]:
"""
11. 分類に貢献した特徴領域の可視化（Grad-CAM）

このコードでは、GradCAMクラスを定義し、
モデルの特定の層の出力と勾配をフックしてGrad-CAMマップを生成する。
その後、生成されたマップを元画像に重ねて表示する。
target_layerは可視化したい層を指定（例ではfeatures[4]を使用）。

# モデル内の層名表示は以下の通り実行する
for name, module in neuralnet.named_modules():
    print(name, ":", module)
"""

import torch.nn.functional as F
from torchvision.transforms.functional import normalize
import random
import cv2

def denormalize(image):
    # [-1, 1] の範囲から [0, 1] の範囲に変換する関数
    return image * 0.5 + 0.5

class GradCAM:
    def __init__(self, model, target_layer):
        # Grad-CAMクラスの初期化
        # model: PyTorchモデル
        # target_layer: 可視化したい層
        self.model = model
        self.target_layer = target_layer
        self.gradients = None  # 勾配を保存する変数
        self.activations = None  # アクティベーションを保存する変数

        # フォワードパス時にアクティベーションを取得するフックを登録
        target_layer.register_forward_hook(self._forward_hook)
        # バックプロパゲーション時に勾配を取得するフックを登録
        target_layer.register_backward_hook(self._backward_hook)

    def _forward_hook(self, module, input, output):
        # フォワードパス時に呼び出されるフック関数
        self.activations = output

    def _backward_hook(self, module, grad_input, grad_output):
        # バックプロパゲーション時に呼び出されるフック関数
        self.gradients = grad_output[0]

    def generate(self, input_image, class_idx):
        # Grad-CAMマップを生成する関数
        # input_image: 入力画像 (1枚)
        # class_idx: 対象のクラスインデックス (Noneの場合は自動で最大の出力クラスを選択)

        self.model.eval()  # モデルを評価モードに設定

        # フォワードパスを実行
        output = self.model(input_image)
        if class_idx is None:
            # クラスインデックスを出力の最大値から自動選択
            class_idx = output.argmax(dim=1).item()

        # 出力の対象クラスに対してバックプロパゲーションを実行
        self.model.zero_grad()
        output[:, class_idx].backward()

        # 勾配をチャンネルごとに平均化
        pooled_gradients = torch.mean(self.gradients, dim=(0, 2, 3))
        activations = self.activations[0]  # 対応するアクティベーションを取得

        # アクティベーションに勾配の影響を掛け合わせる
        for i in range(len(pooled_gradients)):
            activations[i, :, :] *= pooled_gradients[i]

        # チャンネル方向に平均化してヒートマップを生成
        heatmap = torch.mean(activations, dim=0).cpu().detach().numpy()
        heatmap = np.maximum(heatmap, 0)  # 負の値を0にクリップ
        heatmap /= np.max(heatmap)  # 正規化して [0, 1] の範囲に変換
        return heatmap

def show_gradcam(model, gradcam, loader, device, num_samples=5):
    model.eval()
    all_images, all_labels = next(iter(loader))
    indices = random.sample(range(len(all_images)), num_samples)
    images, labels = all_images[indices].to(device), all_labels[indices]

    fig, axes = plt.subplots(num_samples, 2, figsize=(10, 15))
    for i in range(num_samples):
        image = images[i].unsqueeze(0)
        # GradCAM地図をつくる
        heatmap = gradcam.generate(image, class_idx=None)

        # Predicted classを得る
        with torch.no_grad():
            pred = model(image).argmax(dim=1).item()

        # heatmapをリサイズする
        heatmap = np.uint8(255 * heatmap)
        heatmap = cv2.resize(heatmap, (32, 32))

        # 画像処理
        image_np = denormalize(images[i].cpu()).numpy().transpose(1, 2, 0)

        # OpenCVのためにBGRへコンバート
        image_bgr = cv2.cvtColor(np.uint8(image_np * 255), cv2.COLOR_RGB2BGR)

        # color mapを適用
        heatmap_img = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

        # Overlayをつくる
        overlay = cv2.addWeighted(
            image_bgr, 0.6,
            heatmap_img, 0.4,
            0
        )

        # matplotlibのためにRGBにもどす
        overlay_rgb = cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)

        # original imageを表示
        axes[i, 0].imshow(image_np)
        axes[i, 0].axis('off')
        axes[i, 0].set_title(f"Original (Label: {labels[i].item()}, Pred: {pred})")

        # GradCAMを表示
        axes[i, 1].imshow(overlay_rgb)
        axes[i, 1].axis('off')
        axes[i, 1].set_title("Grad-CAM")

    plt.tight_layout()
    plt.show()

# Grad-CAMを初期化 (使用する層を指定)
target_layer = neuralnet.features[4]  # featuresの5番目の層を指定
gradcam = GradCAM(neuralnet, target_layer)

# Grad-CAMを使用して可視化を実行
show_gradcam(neuralnet, gradcam, test_loader, device, num_samples=5)